In [ ]:
"""
============================================================================
VERSİYON 1: NORMALIZATION LAG ÖNCESİ
============================================================================
Hipotez: Normalization indicator'lar hesaplandıktan hemen sonra yapılmalı,
         lag'den ÖNCE!
============================================================================
"""

import sys
import subprocess
print("📦 VERSİYON 1: Normalization LAG öncesi")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                      "yfinance", "ta", "scikit-learn", "pandas", "numpy"])

import yfinance as yf
import pandas as pd
import numpy as np
import ta
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Kurulum tamamlandı!\n")

# ============================================================================
# LEVEL 1: VERİ ÇEKME
# ============================================================================
print("="*80)
print("LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)")
print("="*80)

tickers = {
    'KSE100': '^KSE',
    'KOSPI': '^KS11',
    'Nikkei225': '^N225',
    'SZSE': '000001.SS'
}

all_data = {}
for name, ticker in tickers.items():
    print(f"\n📊 {name} ({ticker})...", end=" ")
    try:
        data = yf.download(ticker, start="2011-01-01", end="2020-09-27",
                          progress=False, auto_adjust=True)

        if data.empty:
            print("❌ VERİ YOK!")
            continue

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        data = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        data = data.dropna()

        all_data[name] = data
        print(f"✅ {len(data)} gün")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(all_data)} borsa çekildi")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 2: TEKNİK GÖSTERGELER (Table 1)
# ============================================================================
print("="*80)
print("LEVEL 2: 15 TEKNİK GÖSTERGE (Table 1)")
print("="*80)

def hesapla_teknik_gostergeler(df):
    """Makalenin Table 1 formüllerine göre"""
    df = df.copy()

    high = df['High'].squeeze()
    low = df['Low'].squeeze()
    close = df['Close'].squeeze()

    # 1-2. Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(high, low, close,
                                             window=14, smooth_window=3)
    df['Stochastic_K'] = stoch.stoch()
    df['Stochastic_D'] = stoch.stoch_signal()

    # 3. ROC (10 period)
    df['ROC'] = ta.momentum.ROCIndicator(close, window=10).roc()

    # 4. Williams %R (14 period)
    df['Williams_R'] = ta.momentum.WilliamsRIndicator(high, low, close,
                                                       lbp=14).williams_r()

    # 5. Momentum (4 period: C_t - C_{t-4})
    df['Momentum'] = close.diff(4)

    # 6. Disparity 5
    ma5 = close.rolling(5).mean()
    df['Disparity_5'] = np.where(ma5 != 0, (close / ma5) * 100, 100)

    # 7. Disparity 14 (makalede 15 yazıyor ama 14 kullanılmış)
    ma14 = close.rolling(14).mean()
    df['Disparity_14'] = np.where(ma14 != 0, (close / ma14) * 100, 100)

    # 8. OSCP (Price Oscillator)
    ma10 = close.rolling(10).mean()
    df['OSCP'] = np.where(ma5 != 0, ((ma5 - ma10) / ma5), 0)

    # 9. CCI (20 period)
    df['CCI'] = ta.trend.CCIIndicator(high, low, close, window=20).cci()

    # 10. RSI (14 period)
    df['RSI'] = ta.momentum.RSIIndicator(close, window=14).rsi()

    # 11-15. Pivot Points (Table 1 formulas)
    prev_high = high.shift(1)
    prev_low = low.shift(1)
    prev_close = close.shift(1)

    df['Pivot_Point'] = (prev_high + prev_low + prev_close) / 3
    df['S1'] = (df['Pivot_Point'] * 2) - prev_high
    df['S2'] = df['Pivot_Point'] - (prev_high - prev_low)
    df['R1'] = (df['Pivot_Point'] * 2) - prev_low
    df['R2'] = df['Pivot_Point'] + (prev_high - prev_low)

    # Inf/NaN temizleme
    df = df.replace([np.inf, -np.inf], np.nan)

    return df

all_data_indicators = {}
for name, data in all_data.items():
    print(f"\n{name}...", end=" ")
    try:
        result = hesapla_teknik_gostergeler(data)
        all_data_indicators[name] = result
        print(f"✅ {len(result)} satır")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ Göstergeler hesaplandı")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 3: VERİ HAZIRLAMA - NORMALIZATION LAG ÖNCESİ
# ============================================================================
print("="*80)
print("LEVEL 3: VERİ HAZIRLAMA (NORMALIZATION LAG ÖNCESİ)")
print("="*80)

def veri_hazirla(df, test_ratio=0.2):
    """
    🎯 VERSİYON 1: Normalization LAG öncesi

    1. Indicator'ları hesapla
    2. NORMALIZE ET (tüm veri seti)
    3. Lag uygula
    4. Train/Test split
    """
    df = df.copy()

    features = ['Stochastic_K', 'Stochastic_D', 'ROC', 'Williams_R',
                'Momentum', 'Disparity_5', 'Disparity_14', 'OSCP',
                'CCI', 'RSI', 'Pivot_Point', 'S1', 'S2', 'R1', 'R2']

    # 1️⃣ Target oluştur (forward-looking)
    df['Next_Close'] = df['Close'].shift(-1)
    df['Target'] = (df['Next_Close'] > df['Close']).astype(int)
    df = df.iloc[:-1].copy()

    # 2️⃣ NaN temizle
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=features + ['Target'])

    # 3️⃣ NORMALIZATION ÖNCE (tüm veri seti - data leakage var ama makale böyle yapıyor olabilir)
    scaler = MinMaxScaler()
    df[features] = scaler.fit_transform(df[features])

    # 4️⃣ ŞİMDİ LAG UYGULA
    lagged_features = []
    for feat in features:
        lagged_col = f'{feat}_lag1'
        df[lagged_col] = df[feat].shift(1)
        lagged_features.append(lagged_col)

    # 5️⃣ Lag yüzünden ilk satırı drop
    df = df.dropna(subset=lagged_features)

    X = df[lagged_features].copy()
    y = df['Target'].copy()

    print(f" Veri: {len(X)} satır | Up: {y.mean()*100:.1f}%")

    # 6️⃣ Zaman bazlı split
    n_train = int(len(X) * (1 - test_ratio))
    X_train = X.iloc[:n_train].copy()
    X_test = X.iloc[n_train:].copy()
    y_train = y.iloc[:n_train].values
    y_test = y.iloc[n_train:].values

    print(f" Train: {len(X_train)} | Test: {len(X_test)}")

    return X_train, X_test, y_train, y_test

prepared_data = {}
for name, data in all_data_indicators.items():
    print(f"\n{name}:")
    try:
        X_train, X_test, y_train, y_test = veri_hazirla(data)
        prepared_data[name] = {
            'X_train': X_train, 'X_test': X_test,
            'y_train': y_train, 'y_test': y_test
        }
    except Exception as e:
        print(f" ❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(prepared_data)} borsa hazır")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 4: SVM SADECE LINEAR (HIZLI TEST)
# ============================================================================
print("="*80)
print("LEVEL 4: SVM LINEAR KERNEL (Hızlı Test)")
print("="*80)

for name in ['KOSPI']:  # Sadece KOSPI test
    print(f"\n{name}:")
    print(f" {'-'*70}")

    data = prepared_data[name]

    try:
        param_grid = {
            'C': [0.001, 0.01, 0.1, 1, 4, 10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
        }

        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        svm = SVC(kernel='linear', max_iter=50000, random_state=42)
        grid = GridSearchCV(svm, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=0)
        grid.fit(data['X_train'], data['y_train'])

        best_model = grid.best_estimator_
        y_pred = best_model.predict(data['X_test'])

        acc = accuracy_score(data['y_test'], y_pred)
        f1 = f1_score(data['y_test'], y_pred, zero_division=0)

        print(f" Best C: {grid.best_params_['C']}")
        print(f" CV Score: {grid.best_score_:.4f}")
        print(f" Test Acc: {acc:.4f} | F1: {f1:.4f}")

    except Exception as e:
        print(f" ❌ {e}")

print("\n" + "="*80)
print("✅ VERSİYON 1 TEST TAMAMLANDI")
print("="*80)

📦 VERSİYON 1: Normalization LAG öncesi
✅ Kurulum tamamlandı!

LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)

📊 KSE100 (^KSE)... ✅ 2346 gün

📊 KOSPI (^KS11)... ✅ 2397 gün

📊 Nikkei225 (^N225)... ✅ 2382 gün

📊 SZSE (000001.SS)... ✅ 2366 gün

✅ 4 borsa çekildi

LEVEL 2: 15 TEKNİK GÖSTERGE (Table 1)

KSE100... ✅ 2346 satır

KOSPI... ✅ 2397 satır

Nikkei225... ✅ 2382 satır

SZSE... ✅ 2366 satır

✅ Göstergeler hesaplandı

LEVEL 3: VERİ HAZIRLAMA (NORMALIZATION LAG ÖNCESİ)

KSE100:
 Veri: 2325 satır | Up: 53.5%
 Train: 1860 | Test: 465

KOSPI:
 Veri: 2376 satır | Up: 52.4%
 Train: 1900 | Test: 476

Nikkei225:
 Veri: 2361 satır | Up: 53.1%
 Train: 1888 | Test: 473

SZSE:
 Veri: 2345 satır | Up: 52.8%
 Train: 1876 | Test: 469

✅ 4 borsa hazır

LEVEL 4: SVM LINEAR KERNEL (Hızlı Test)

KOSPI:
 ----------------------------------------------------------------------
 Best C: 0.001
 CV Score: 0.5137
 Test Acc: 0.5630 | F1: 0.7204

✅ VERSİYON 1 TEST TAMAMLANDI


In [ ]:
"""
============================================================================
VERSİYON 2: TARGET SAME-DAY
============================================================================
Hipotez: Target aynı gün içinde olmalı!
         Feature: T-1 günü indicator'ları
         Target: T günü Close > T günü Open mi? (veya T > T-1)
============================================================================
"""

import sys
import subprocess
print("📦 VERSİYON 2: Target Same-Day")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                      "yfinance", "ta", "scikit-learn", "pandas", "numpy"])

import yfinance as yf
import pandas as pd
import numpy as np
import ta
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Kurulum tamamlandı!\n")

# ============================================================================
# LEVEL 1: VERİ ÇEKME
# ============================================================================
print("="*80)
print("LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)")
print("="*80)

tickers = {
    'KSE100': '^KSE',
    'KOSPI': '^KS11',
    'Nikkei225': '^N225',
    'SZSE': '000001.SS'
}

all_data = {}
for name, ticker in tickers.items():
    print(f"\n📊 {name} ({ticker})...", end=" ")
    try:
        data = yf.download(ticker, start="2011-01-01", end="2020-09-27",
                          progress=False, auto_adjust=True)

        if data.empty:
            print("❌ VERİ YOK!")
            continue

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        data = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        data = data.dropna()

        all_data[name] = data
        print(f"✅ {len(data)} gün")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(all_data)} borsa çekildi")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 2: TEKNİK GÖSTERGELER (Table 1)
# ============================================================================
print("="*80)
print("LEVEL 2: 15 TEKNİK GÖSTERGE (Table 1)")
print("="*80)

def hesapla_teknik_gostergeler(df):
    """Makalenin Table 1 formüllerine göre"""
    df = df.copy()

    high = df['High'].squeeze()
    low = df['Low'].squeeze()
    close = df['Close'].squeeze()

    # 1-2. Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(high, low, close,
                                             window=14, smooth_window=3)
    df['Stochastic_K'] = stoch.stoch()
    df['Stochastic_D'] = stoch.stoch_signal()

    # 3. ROC (10 period)
    df['ROC'] = ta.momentum.ROCIndicator(close, window=10).roc()

    # 4. Williams %R (14 period)
    df['Williams_R'] = ta.momentum.WilliamsRIndicator(high, low, close,
                                                       lbp=14).williams_r()

    # 5. Momentum (4 period: C_t - C_{t-4})
    df['Momentum'] = close.diff(4)

    # 6. Disparity 5
    ma5 = close.rolling(5).mean()
    df['Disparity_5'] = np.where(ma5 != 0, (close / ma5) * 100, 100)

    # 7. Disparity 14 (makalede 15 yazıyor ama 14 kullanılmış)
    ma14 = close.rolling(14).mean()
    df['Disparity_14'] = np.where(ma14 != 0, (close / ma14) * 100, 100)

    # 8. OSCP (Price Oscillator)
    ma10 = close.rolling(10).mean()
    df['OSCP'] = np.where(ma5 != 0, ((ma5 - ma10) / ma5), 0)

    # 9. CCI (20 period)
    df['CCI'] = ta.trend.CCIIndicator(high, low, close, window=20).cci()

    # 10. RSI (14 period)
    df['RSI'] = ta.momentum.RSIIndicator(close, window=14).rsi()

    # 11-15. Pivot Points (Table 1 formulas)
    prev_high = high.shift(1)
    prev_low = low.shift(1)
    prev_close = close.shift(1)

    df['Pivot_Point'] = (prev_high + prev_low + prev_close) / 3
    df['S1'] = (df['Pivot_Point'] * 2) - prev_high
    df['S2'] = df['Pivot_Point'] - (prev_high - prev_low)
    df['R1'] = (df['Pivot_Point'] * 2) - prev_low
    df['R2'] = df['Pivot_Point'] + (prev_high - prev_low)

    # Inf/NaN temizleme
    df = df.replace([np.inf, -np.inf], np.nan)

    return df

all_data_indicators = {}
for name, data in all_data.items():
    print(f"\n{name}...", end=" ")
    try:
        result = hesapla_teknik_gostergeler(data)
        all_data_indicators[name] = result
        print(f"✅ {len(result)} satır")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ Göstergeler hesaplandı")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 3: VERİ HAZIRLAMA - TARGET SAME-DAY
# ============================================================================
print("="*80)
print("LEVEL 3: VERİ HAZIRLAMA (TARGET SAME-DAY)")
print("="*80)

def veri_hazirla(df, test_ratio=0.2):
    """
    🎯 VERSİYON 2: Target same-day (T günü)

    Feature: T-1 günü indicator'ları
    Target: T günü Close > T-1 günü Close mi?
    """
    df = df.copy()

    features = ['Stochastic_K', 'Stochastic_D', 'ROC', 'Williams_R',
                'Momentum', 'Disparity_5', 'Disparity_14', 'OSCP',
                'CCI', 'RSI', 'Pivot_Point', 'S1', 'S2', 'R1', 'R2']

    # 1️⃣ Target: T günü > T-1 günü mi?
    df['Target'] = (df['Close'] > df['Close'].shift(1)).astype(int)

    # 2️⃣ Feature'ları lag'le (T-1)
    lagged_features = []
    for feat in features:
        lagged_col = f'{feat}_lag1'
        df[lagged_col] = df[feat].shift(1)
        lagged_features.append(lagged_col)

    # 3️⃣ NaN temizle
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=lagged_features + ['Target'])

    X = df[lagged_features].copy()
    y = df['Target'].copy()

    print(f" Veri: {len(X)} satır | Up: {y.mean()*100:.1f}%")

    # 4️⃣ Zaman bazlı split
    n_train = int(len(X) * (1 - test_ratio))
    X_train = X.iloc[:n_train].copy()
    X_test = X.iloc[n_train:].copy()
    y_train = y.iloc[:n_train].copy()
    y_test = y.iloc[n_train:].copy()

    # 5️⃣ Normalization (train/test ayrı ayrı)
    scaler = MinMaxScaler()
    X_train_scaled = pd.DataFrame(
        scaler.fit_transform(X_train),
        columns=lagged_features,
        index=X_train.index
    )
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test),
        columns=lagged_features,
        index=X_test.index
    )

    print(f" Train: {len(X_train_scaled)} | Test: {len(X_test_scaled)}")

    return X_train_scaled, X_test_scaled, y_train.values, y_test.values

prepared_data = {}
for name, data in all_data_indicators.items():
    print(f"\n{name}:")
    try:
        X_train, X_test, y_train, y_test = veri_hazirla(data)
        prepared_data[name] = {
            'X_train': X_train, 'X_test': X_test,
            'y_train': y_train, 'y_test': y_test
        }
    except Exception as e:
        print(f" ❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(prepared_data)} borsa hazır")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 4: SVM SADECE LINEAR (HIZLI TEST)
# ============================================================================
print("="*80)
print("LEVEL 4: SVM LINEAR KERNEL (Hızlı Test)")
print("="*80)

for name in ['KOSPI']:  # Sadece KOSPI test
    print(f"\n{name}:")
    print(f" {'-'*70}")

    data = prepared_data[name]

    try:
        param_grid = {
            'C': [0.001, 0.01, 0.1, 1, 4, 10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
        }

        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        svm = SVC(kernel='linear', max_iter=50000, random_state=42)
        grid = GridSearchCV(svm, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=0)
        grid.fit(data['X_train'], data['y_train'])

        best_model = grid.best_estimator_
        y_pred = best_model.predict(data['X_test'])

        acc = accuracy_score(data['y_test'], y_pred)
        f1 = f1_score(data['y_test'], y_pred, zero_division=0)

        print(f" Best C: {grid.best_params_['C']}")
        print(f" CV Score: {grid.best_score_:.4f}")
        print(f" Test Acc: {acc:.4f} | F1: {f1:.4f}")

    except Exception as e:
        print(f" ❌ {e}")

print("\n" + "="*80)
print("✅ VERSİYON 2 TEST TAMAMLANDI")
print("="*80)

📦 VERSİYON 2: Target Same-Day
✅ Kurulum tamamlandı!

LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)

📊 KSE100 (^KSE)... ✅ 2346 gün

📊 KOSPI (^KS11)... ✅ 2397 gün

📊 Nikkei225 (^N225)... ✅ 2382 gün

📊 SZSE (000001.SS)... ✅ 2366 gün

✅ 4 borsa çekildi

LEVEL 2: 15 TEKNİK GÖSTERGE (Table 1)

KSE100... ✅ 2346 satır

KOSPI... ✅ 2397 satır

Nikkei225... ✅ 2382 satır

SZSE... ✅ 2366 satır

✅ Göstergeler hesaplandı

LEVEL 3: VERİ HAZIRLAMA (TARGET SAME-DAY)

KSE100:
 Veri: 2326 satır | Up: 53.5%
 Train: 1860 | Test: 466

KOSPI:
 Veri: 2377 satır | Up: 52.3%
 Train: 1901 | Test: 476

Nikkei225:
 Veri: 2362 satır | Up: 53.1%
 Train: 1889 | Test: 473

SZSE:
 Veri: 2346 satır | Up: 52.9%
 Train: 1876 | Test: 470

✅ 4 borsa hazır

LEVEL 4: SVM LINEAR KERNEL (Hızlı Test)

KOSPI:
 ----------------------------------------------------------------------
 Best C: 900
 CV Score: 0.5187
 Test Acc: 0.5147 | F1: 0.5295

✅ VERSİYON 2 TEST TAMAMLANDI


In [ ]:
"""
============================================================================
VERSİYON 3: PIVOT POINTS DOUBLE-LAG DÜZELTMESİ
============================================================================
Hipotez: Pivot Points zaten shift(1) ile hesaplanıyor,
         sonra bir de biz lag yapınca double-lag oluyor!
============================================================================
"""

import sys
import subprocess
print("📦 VERSİYON 3: Pivot Points Double-Lag Fix")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                      "yfinance", "ta", "scikit-learn", "pandas", "numpy"])

import yfinance as yf
import pandas as pd
import numpy as np
import ta
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Kurulum tamamlandı!\n")

# ============================================================================
# LEVEL 1: VERİ ÇEKME
# ============================================================================
print("="*80)
print("LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)")
print("="*80)

tickers = {
    'KSE100': '^KSE',
    'KOSPI': '^KS11',
    'Nikkei225': '^N225',
    'SZSE': '000001.SS'
}

all_data = {}
for name, ticker in tickers.items():
    print(f"\n📊 {name} ({ticker})...", end=" ")
    try:
        data = yf.download(ticker, start="2011-01-01", end="2020-09-27",
                          progress=False, auto_adjust=True)

        if data.empty:
            print("❌ VERİ YOK!")
            continue

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        data = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        data = data.dropna()

        all_data[name] = data
        print(f"✅ {len(data)} gün")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(all_data)} borsa çekildi")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 2: TEKNİK GÖSTERGELER - PIVOT POINTS DÜZELTİLDİ
# ============================================================================
print("="*80)
print("LEVEL 2: 15 TEKNİK GÖSTERGE (Pivot Points Düzeltildi)")
print("="*80)

def hesapla_teknik_gostergeler(df):
    """Makalenin Table 1 formüllerine göre - Pivot Points düzeltildi"""
    df = df.copy()

    high = df['High'].squeeze()
    low = df['Low'].squeeze()
    close = df['Close'].squeeze()

    # 1-2. Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(high, low, close,
                                             window=14, smooth_window=3)
    df['Stochastic_K'] = stoch.stoch()
    df['Stochastic_D'] = stoch.stoch_signal()

    # 3. ROC (10 period)
    df['ROC'] = ta.momentum.ROCIndicator(close, window=10).roc()

    # 4. Williams %R (14 period)
    df['Williams_R'] = ta.momentum.WilliamsRIndicator(high, low, close,
                                                       lbp=14).williams_r()

    # 5. Momentum (4 period: C_t - C_{t-4})
    df['Momentum'] = close.diff(4)

    # 6. Disparity 5
    ma5 = close.rolling(5).mean()
    df['Disparity_5'] = np.where(ma5 != 0, (close / ma5) * 100, 100)

    # 7. Disparity 14 (makalede 15 yazıyor ama 14 kullanılmış)
    ma14 = close.rolling(14).mean()
    df['Disparity_14'] = np.where(ma14 != 0, (close / ma14) * 100, 100)

    # 8. OSCP (Price Oscillator)
    ma10 = close.rolling(10).mean()
    df['OSCP'] = np.where(ma5 != 0, ((ma5 - ma10) / ma5), 0)

    # 9. CCI (20 period)
    df['CCI'] = ta.trend.CCIIndicator(high, low, close, window=20).cci()

    # 10. RSI (14 period)
    df['RSI'] = ta.momentum.RSIIndicator(close, window=14).rsi()

    # 11-15. Pivot Points - ŞİMDİ SHIFT KULLANMIYORUZ
    # Çünkü sonra zaten lag yapacağız!
    df['Pivot_Point'] = (high + low + close) / 3
    df['S1'] = (df['Pivot_Point'] * 2) - high
    df['S2'] = df['Pivot_Point'] - (high - low)
    df['R1'] = (df['Pivot_Point'] * 2) - low
    df['R2'] = df['Pivot_Point'] + (high - low)

    # Inf/NaN temizleme
    df = df.replace([np.inf, -np.inf], np.nan)

    return df

all_data_indicators = {}
for name, data in all_data.items():
    print(f"\n{name}...", end=" ")
    try:
        result = hesapla_teknik_gostergeler(data)
        all_data_indicators[name] = result
        print(f"✅ {len(result)} satır")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ Göstergeler hesaplandı")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 3: VERİ HAZIRLAMA (Standart)
# ============================================================================
print("="*80)
print("LEVEL 3: VERİ HAZIRLAMA")
print("="*80)

def veri_hazirla(df, test_ratio=0.2):
    """
    🎯 VERSİYON 3: Pivot Points double-lag düzeltildi

    Feature: T-1 günü indicator'ları
    Target: T günü → T+1 günü yükseliş mi?
    """
    df = df.copy()

    features = ['Stochastic_K', 'Stochastic_D', 'ROC', 'Williams_R',
                'Momentum', 'Disparity_5', 'Disparity_14', 'OSCP',
                'CCI', 'RSI', 'Pivot_Point', 'S1', 'S2', 'R1', 'R2']

    # 1️⃣ Feature'ları lag'le (T-1)
    lagged_features = []
    for feat in features:
        lagged_col = f'{feat}_lag1'
        df[lagged_col] = df[feat].shift(1)
        lagged_features.append(lagged_col)

    # 2️⃣ Target: T günü → T+1 günü yükseliş mi?
    df['Next_Close'] = df['Close'].shift(-1)
    df['Target'] = (df['Next_Close'] > df['Close']).astype(int)
    df = df.iloc[:-1].copy()

    # 3️⃣ NaN temizle
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=lagged_features + ['Target'])

    X = df[lagged_features].copy()
    y = df['Target'].copy()

    print(f" Veri: {len(X)} satır | Up: {y.mean()*100:.1f}%")

    # 4️⃣ Zaman bazlı split
    n_train = int(len(X) * (1 - test_ratio))
    X_train = X.iloc[:n_train].copy()
    X_test = X.iloc[n_train:].copy()
    y_train = y.iloc[:n_train].copy()
    y_test = y.iloc[n_train:].copy()

    # 5️⃣ Normalization
    scaler = MinMaxScaler()
    X_train_scaled = pd.DataFrame(
        scaler.fit_transform(X_train),
        columns=lagged_features,
        index=X_train.index
    )
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test),
        columns=lagged_features,
        index=X_test.index
    )

    print(f" Train: {len(X_train_scaled)} | Test: {len(X_test_scaled)}")

    return X_train_scaled, X_test_scaled, y_train.values, y_test.values

prepared_data = {}
for name, data in all_data_indicators.items():
    print(f"\n{name}:")
    try:
        X_train, X_test, y_train, y_test = veri_hazirla(data)
        prepared_data[name] = {
            'X_train': X_train, 'X_test': X_test,
            'y_train': y_train, 'y_test': y_test
        }
    except Exception as e:
        print(f" ❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(prepared_data)} borsa hazır")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 4: SVM SADECE LINEAR (HIZLI TEST)
# ============================================================================
print("="*80)
print("LEVEL 4: SVM LINEAR KERNEL (Hızlı Test)")
print("="*80)

for name in ['KOSPI']:  # Sadece KOSPI test
    print(f"\n{name}:")
    print(f" {'-'*70}")

    data = prepared_data[name]

    try:
        param_grid = {
            'C': [0.001, 0.01, 0.1, 1, 4, 10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
        }

        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        svm = SVC(kernel='linear', max_iter=50000, random_state=42)
        grid = GridSearchCV(svm, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=0)
        grid.fit(data['X_train'], data['y_train'])

        best_model = grid.best_estimator_
        y_pred = best_model.predict(data['X_test'])

        acc = accuracy_score(data['y_test'], y_pred)
        f1 = f1_score(data['y_test'], y_pred, zero_division=0)

        print(f" Best C: {grid.best_params_['C']}")
        print(f" CV Score: {grid.best_score_:.4f}")
        print(f" Test Acc: {acc:.4f} | F1: {f1:.4f}")

    except Exception as e:
        print(f" ❌ {e}")

print("\n" + "="*80)
print("✅ VERSİYON 3 TEST TAMAMLANDI")
print("="*80)

📦 VERSİYON 3: Pivot Points Double-Lag Fix
✅ Kurulum tamamlandı!

LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)

📊 KSE100 (^KSE)... ✅ 2346 gün

📊 KOSPI (^KS11)... ✅ 2397 gün

📊 Nikkei225 (^N225)... ✅ 2382 gün

📊 SZSE (000001.SS)... ✅ 2366 gün

✅ 4 borsa çekildi

LEVEL 2: 15 TEKNİK GÖSTERGE (Pivot Points Düzeltildi)

KSE100... ✅ 2346 satır

KOSPI... ✅ 2397 satır

Nikkei225... ✅ 2382 satır

SZSE... ✅ 2366 satır

✅ Göstergeler hesaplandı

LEVEL 3: VERİ HAZIRLAMA

KSE100:
 Veri: 2325 satır | Up: 53.5%
 Train: 1860 | Test: 465

KOSPI:
 Veri: 2376 satır | Up: 52.4%
 Train: 1900 | Test: 476

Nikkei225:
 Veri: 2361 satır | Up: 53.1%
 Train: 1888 | Test: 473

SZSE:
 Veri: 2345 satır | Up: 52.8%
 Train: 1876 | Test: 469

✅ 4 borsa hazır

LEVEL 4: SVM LINEAR KERNEL (Hızlı Test)

KOSPI:
 ----------------------------------------------------------------------
 Best C: 0.001
 CV Score: 0.5137
 Test Acc: 0.5630 | F1: 0.7204

✅ VERSİYON 3 TEST TAMAMLANDI


In [ ]:
"""
============================================================================
VERSİYON 1 FULL: NORMALIZATION LAG ÖNCESİ - TÜM SVM KERNELS
============================================================================
Hipotez: Makale normalization'ı LAG öncesi yapıyor (data leakage var)
         Bu yüzden yüksek accuracy elde ediyorlar!
============================================================================
"""

import sys
import subprocess
print("📦 VERSİYON 1 FULL: Normalization LAG öncesi - TÜM SVM")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                      "yfinance", "ta", "scikit-learn", "pandas", "numpy"])

import yfinance as yf
import pandas as pd
import numpy as np
import ta
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Kurulum tamamlandı!\n")

# ============================================================================
# LEVEL 1: VERİ ÇEKME
# ============================================================================
print("="*80)
print("LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)")
print("="*80)

tickers = {
    'KSE100': '^KSE',
    'KOSPI': '^KS11',
    'Nikkei225': '^N225',
    'SZSE': '000001.SS'
}

all_data = {}
for name, ticker in tickers.items():
    print(f"\n📊 {name} ({ticker})...", end=" ")
    try:
        data = yf.download(ticker, start="2011-01-01", end="2020-09-27",
                          progress=False, auto_adjust=True)

        if data.empty:
            print("❌ VERİ YOK!")
            continue

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        data = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        data = data.dropna()

        all_data[name] = data
        print(f"✅ {len(data)} gün")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(all_data)} borsa çekildi")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 2: TEKNİK GÖSTERGELER (Table 1)
# ============================================================================
print("="*80)
print("LEVEL 2: 15 TEKNİK GÖSTERGE (Table 1)")
print("="*80)

def hesapla_teknik_gostergeler(df):
    """Makalenin Table 1 formüllerine göre"""
    df = df.copy()

    high = df['High'].squeeze()
    low = df['Low'].squeeze()
    close = df['Close'].squeeze()

    # 1-2. Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(high, low, close,
                                             window=14, smooth_window=3)
    df['Stochastic_K'] = stoch.stoch()
    df['Stochastic_D'] = stoch.stoch_signal()

    # 3. ROC (10 period)
    df['ROC'] = ta.momentum.ROCIndicator(close, window=10).roc()

    # 4. Williams %R (14 period)
    df['Williams_R'] = ta.momentum.WilliamsRIndicator(high, low, close,
                                                       lbp=14).williams_r()

    # 5. Momentum (4 period: C_t - C_{t-4})
    df['Momentum'] = close.diff(4)

    # 6. Disparity 5
    ma5 = close.rolling(5).mean()
    df['Disparity_5'] = np.where(ma5 != 0, (close / ma5) * 100, 100)

    # 7. Disparity 14 (makalede 15 yazıyor ama 14 kullanılmış)
    ma14 = close.rolling(14).mean()
    df['Disparity_14'] = np.where(ma14 != 0, (close / ma14) * 100, 100)

    # 8. OSCP (Price Oscillator)
    ma10 = close.rolling(10).mean()
    df['OSCP'] = np.where(ma5 != 0, ((ma5 - ma10) / ma5), 0)

    # 9. CCI (20 period)
    df['CCI'] = ta.trend.CCIIndicator(high, low, close, window=20).cci()

    # 10. RSI (14 period)
    df['RSI'] = ta.momentum.RSIIndicator(close, window=14).rsi()

    # 11-15. Pivot Points (Table 1 formulas)
    prev_high = high.shift(1)
    prev_low = low.shift(1)
    prev_close = close.shift(1)

    df['Pivot_Point'] = (prev_high + prev_low + prev_close) / 3
    df['S1'] = (df['Pivot_Point'] * 2) - prev_high
    df['S2'] = df['Pivot_Point'] - (prev_high - prev_low)
    df['R1'] = (df['Pivot_Point'] * 2) - prev_low
    df['R2'] = df['Pivot_Point'] + (prev_high - prev_low)

    # Inf/NaN temizleme
    df = df.replace([np.inf, -np.inf], np.nan)

    return df

all_data_indicators = {}
for name, data in all_data.items():
    print(f"\n{name}...", end=" ")
    try:
        result = hesapla_teknik_gostergeler(data)
        all_data_indicators[name] = result
        print(f"✅ {len(result)} satır")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ Göstergeler hesaplandı")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 3: VERİ HAZIRLAMA - NORMALIZATION LAG ÖNCESİ
# ============================================================================
print("="*80)
print("LEVEL 3: VERİ HAZIRLAMA (NORMALIZATION LAG ÖNCESİ)")
print("="*80)

def veri_hazirla(df, test_ratio=0.2):
    """
    🎯 VERSİYON 1: Normalization LAG öncesi (DATA LEAKAGE VAR!)

    1. Indicator'ları hesapla
    2. NORMALIZE ET (tüm veri seti - data leakage!)
    3. Lag uygula
    4. Train/Test split
    """
    df = df.copy()

    features = ['Stochastic_K', 'Stochastic_D', 'ROC', 'Williams_R',
                'Momentum', 'Disparity_5', 'Disparity_14', 'OSCP',
                'CCI', 'RSI', 'Pivot_Point', 'S1', 'S2', 'R1', 'R2']

    # 1️⃣ Target oluştur (forward-looking)
    df['Next_Close'] = df['Close'].shift(-1)
    df['Target'] = (df['Next_Close'] > df['Close']).astype(int)
    df = df.iloc[:-1].copy()

    # 2️⃣ NaN temizle
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=features + ['Target'])

    # 3️⃣ ⚠️ NORMALIZATION ÖNCE (tüm veri seti - data leakage!)
    scaler = MinMaxScaler()
    df[features] = scaler.fit_transform(df[features])

    # 4️⃣ ŞİMDİ LAG UYGULA
    lagged_features = []
    for feat in features:
        lagged_col = f'{feat}_lag1'
        df[lagged_col] = df[feat].shift(1)
        lagged_features.append(lagged_col)

    # 5️⃣ Lag yüzünden ilk satırı drop
    df = df.dropna(subset=lagged_features)

    X = df[lagged_features].copy()
    y = df['Target'].copy()

    print(f" Veri: {len(X)} satır | Up: {y.mean()*100:.1f}%")

    # 6️⃣ Zaman bazlı split
    n_train = int(len(X) * (1 - test_ratio))
    X_train = X.iloc[:n_train].copy()
    X_test = X.iloc[n_train:].copy()
    y_train = y.iloc[:n_train].values
    y_test = y.iloc[n_train:].values

    print(f" Train: {len(X_train)} | Test: {len(X_test)}")

    return X_train, X_test, y_train, y_test

prepared_data = {}
for name, data in all_data_indicators.items():
    print(f"\n{name}:")
    try:
        X_train, X_test, y_train, y_test = veri_hazirla(data)
        prepared_data[name] = {
            'X_train': X_train, 'X_test': X_test,
            'y_train': y_train, 'y_test': y_test
        }
    except Exception as e:
        print(f" ❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(prepared_data)} borsa hazır")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 4: SVM TÜM KERNELS (Linear, RBF, Poly)
# ============================================================================
print("="*80)
print("LEVEL 4: SVM TÜM KERNELS (10-Fold CV + Grid Search)")
print("="*80)

def grid_search_svm(X_train, y_train, kernel='linear', n_folds=10):
    """Makalenin Figures 3-6'daki exact aralıklar"""

    if kernel == 'linear':
        param_grid = {
            'C': [0.001, 0.01, 0.1, 1, 4, 10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
        }


    cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    svm = SVC(kernel=kernel, max_iter=50000, random_state=42)
    grid = GridSearchCV(svm, param_grid, cv=cv, scoring='accuracy',
                       n_jobs=-1, verbose=0)
    grid.fit(X_train, y_train)

    return grid.best_estimator_, grid.best_params_, grid.best_score_

svm_results = {}

for name in prepared_data.keys():
    print(f"\n{'='*80}")
    print(f"{name}")
    print(f"{'='*80}")

    data = prepared_data[name]
    svm_results[name] = {}

    for kernel in ['linear', 'rbf', 'poly']:
        print(f"\n 🔍 {kernel.upper()} Kernel:")
        print(f" {'-'*70}")

        try:
            best_model, best_params, cv_score = grid_search_svm(
                data['X_train'], data['y_train'], kernel=kernel
            )

            y_pred = best_model.predict(data['X_test'])

            acc = accuracy_score(data['y_test'], y_pred)
            prec = precision_score(data['y_test'], y_pred, zero_division=0)
            rec = recall_score(data['y_test'], y_pred, zero_division=0)
            f1 = f1_score(data['y_test'], y_pred, zero_division=0)

            svm_results[name][kernel] = {
                'model': best_model,
                'params': best_params,
                'cv_score': cv_score,
                'acc': acc,
                'precision': prec,
                'recall': rec,
                'f1': f1
            }

            print(f" Best Params: {best_params}")
            print(f" CV Score: {cv_score:.4f}")
            print(f" Test Acc: {acc:.4f} | F1: {f1:.4f}")

        except Exception as e:
            print(f" ❌ {e}")
            svm_results[name][kernel] = None

print(f"\n{'='*80}")
print("✅ SVM tamamlandı")
print(f"{'='*80}\n")

# ============================================================================
# SONUÇLAR (Table 11 Format - Makale Karşılaştırması)
# ============================================================================
print("\n" + "="*80)
print(" "*20 + "📊 FINAL SONUÇLAR - SVM")
print(" "*15 + "(Table 11 Makale Karşılaştırması)")
print("="*80)

# Her borsa için sonuçlar
for name in prepared_data.keys():
    print(f"\n{'='*80}")
    print(f" {name}")
    print(f"{'='*80}")

    print(f"\n {'Kernel':<15} {'Accuracy':<12} {'F-Score':<12} {'Best Params'}")
    print(f" {'-'*70}")

    for kernel in ['linear', 'rbf', 'poly']:
        if svm_results[name][kernel] is not None:
            r = svm_results[name][kernel]
            params_str = str(r['params'])[:35] + "..." if len(str(r['params'])) > 35 else str(r['params'])
            print(f" {kernel.upper():<15} {r['acc']:<12.4f} {r['f1']:<12.4f} {params_str}")

# Ortalama sonuçlar
print(f"\n{'='*80}")
print(" 📈 ORTALAMA PERFORMANS (4 Borsa)")
print(f"{'='*80}\n")

if len(prepared_data) > 0:
    avg_svm_linear_acc = np.mean([svm_results[n]['linear']['acc'] for n in prepared_data.keys() if svm_results[n]['linear']])
    avg_svm_rbf_acc = np.mean([svm_results[n]['rbf']['acc'] for n in prepared_data.keys() if svm_results[n]['rbf']])
    avg_svm_poly_acc = np.mean([svm_results[n]['poly']['acc'] for n in prepared_data.keys() if svm_results[n]['poly']])

    avg_svm_linear_f1 = np.mean([svm_results[n]['linear']['f1'] for n in prepared_data.keys() if svm_results[n]['linear']])
    avg_svm_rbf_f1 = np.mean([svm_results[n]['rbf']['f1'] for n in prepared_data.keys() if svm_results[n]['rbf']])
    avg_svm_poly_f1 = np.mean([svm_results[n]['poly']['f1'] for n in prepared_data.keys() if svm_results[n]['poly']])

    print(f" {'Kernel':<15} {'Avg Accuracy':<15} {'Avg F-Score':<15}")
    print(f" {'-'*50}")
    print(f" {'Linear':<15} {avg_svm_linear_acc:<15.4f} {avg_svm_linear_f1:<15.4f}")
    print(f" {'RBF':<15} {avg_svm_rbf_acc:<15.4f} {avg_svm_rbf_f1:<15.4f}")
    print(f" {'Poly':<15} {avg_svm_poly_acc:<15.4f} {avg_svm_poly_f1:<15.4f}")

# Makale sonuçları (Table 11)
print(f"\n{'='*80}")
print(" 🎯 MAKALE SONUÇLARI (Table 11 - SVM)")
print(f"{'='*80}\n")
print(" Index       Linear   RBF      Poly")
print(" " + "-"*45)
print(" KSE-100     0.8519   0.7688   0.8438")
print(" KOSPI       0.8022   0.7626   0.7828")
print(" Nikkei 225  0.8022   0.7626   0.7828")
print(" SZSE        0.8998   0.8720   0.8941")

print(f"\n{'='*80}")
print("✅ VERSİYON 1 FULL SVM ANALİZİ TAMAMLANDI")
print(f"{'='*80}\n")

print("💡 ÖNEMLİ NOT:")
print(" ⚠️ Bu versiyonda DATA LEAKAGE var!")
print(" ⚠️ Normalization tüm veri setinde yapıldı (train+test)")
print(" ⚠️ Eğer sonuçlar makaleye yakınsa, makale de aynı hatayı yapıyor olabilir!")

📦 VERSİYON 1 FULL: Normalization LAG öncesi - TÜM SVM
✅ Kurulum tamamlandı!

LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)

📊 KSE100 (^KSE)... ✅ 2346 gün

📊 KOSPI (^KS11)... ✅ 2397 gün

📊 Nikkei225 (^N225)... ✅ 2382 gün

📊 SZSE (000001.SS)... ✅ 2366 gün

✅ 4 borsa çekildi

LEVEL 2: 15 TEKNİK GÖSTERGE (Table 1)

KSE100... ✅ 2346 satır

KOSPI... ✅ 2397 satır

Nikkei225... ✅ 2382 satır

SZSE... ✅ 2366 satır

✅ Göstergeler hesaplandı

LEVEL 3: VERİ HAZIRLAMA (NORMALIZATION LAG ÖNCESİ)

KSE100:
 Veri: 2325 satır | Up: 53.5%
 Train: 1860 | Test: 465

KOSPI:
 Veri: 2376 satır | Up: 52.4%
 Train: 1900 | Test: 476

Nikkei225:
 Veri: 2361 satır | Up: 53.1%
 Train: 1888 | Test: 473

SZSE:
 Veri: 2345 satır | Up: 52.8%
 Train: 1876 | Test: 469

✅ 4 borsa hazır

LEVEL 4: SVM TÜM KERNELS (10-Fold CV + Grid Search)

KSE100

 🔍 LINEAR Kernel:
 ----------------------------------------------------------------------
 Best Params: {'C': 700}
 CV Score: 0.5457
 Test Acc: 0.5527 | F1: 0.5755

 🔍 RBF Kernel

In [ ]:
"""
============================================================================
ADVANCED GRID SEARCH: Two-Stage + Bayesian + Randomized
============================================================================
Hipotez: Best parametre küsüratlı olabilir, akıllı search stratejileri kullanalım!
============================================================================
"""

import sys
import subprocess
print("📦 Advanced Grid Search Yükleniyor...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                      "yfinance", "ta", "scikit-learn", "pandas", "numpy",
                      "scikit-optimize"])  # Bayesian için

import yfinance as yf
import pandas as pd
import numpy as np
import ta
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from scipy.stats import uniform, loguniform
import warnings
warnings.filterwarnings('ignore')

print("✅ Kurulum tamamlandı!\n")

# ============================================================================
# LEVEL 1: VERİ ÇEKME
# ============================================================================
print("="*80)
print("LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)")
print("="*80)

tickers = {
    'KSE100': '^KSE',
    'KOSPI': '^KS11',
    'Nikkei225': '^N225',
    'SZSE': '000001.SS'
}

all_data = {}
for name, ticker in tickers.items():
    print(f"\n📊 {name} ({ticker})...", end=" ")
    try:
        data = yf.download(ticker, start="2011-01-01", end="2020-09-27",
                          progress=False, auto_adjust=True)

        if data.empty:
            print("❌ VERİ YOK!")
            continue

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        data = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        data = data.dropna()

        all_data[name] = data
        print(f"✅ {len(data)} gün")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(all_data)} borsa çekildi")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 2: TEKNİK GÖSTERGELER (Table 1)
# ============================================================================
print("="*80)
print("LEVEL 2: 15 TEKNİK GÖSTERGE (Table 1)")
print("="*80)

def hesapla_teknik_gostergeler(df):
    """Makalenin Table 1 formüllerine göre"""
    df = df.copy()

    high = df['High'].squeeze()
    low = df['Low'].squeeze()
    close = df['Close'].squeeze()

    # 1-2. Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(high, low, close,
                                             window=14, smooth_window=3)
    df['Stochastic_K'] = stoch.stoch()
    df['Stochastic_D'] = stoch.stoch_signal()

    # 3. ROC (10 period)
    df['ROC'] = ta.momentum.ROCIndicator(close, window=10).roc()

    # 4. Williams %R (14 period)
    df['Williams_R'] = ta.momentum.WilliamsRIndicator(high, low, close,
                                                       lbp=14).williams_r()

    # 5. Momentum (4 period: C_t - C_{t-4})
    df['Momentum'] = close.diff(4)

    # 6. Disparity 5
    ma5 = close.rolling(5).mean()
    df['Disparity_5'] = np.where(ma5 != 0, (close / ma5) * 100, 100)

    # 7. Disparity 14
    ma14 = close.rolling(14).mean()
    df['Disparity_14'] = np.where(ma14 != 0, (close / ma14) * 100, 100)

    # 8. OSCP (Price Oscillator)
    ma10 = close.rolling(10).mean()
    df['OSCP'] = np.where(ma5 != 0, ((ma5 - ma10) / ma5), 0)

    # 9. CCI (20 period)
    df['CCI'] = ta.trend.CCIIndicator(high, low, close, window=20).cci()

    # 10. RSI (14 period)
    df['RSI'] = ta.momentum.RSIIndicator(close, window=14).rsi()

    # 11-15. Pivot Points
    prev_high = high.shift(1)
    prev_low = low.shift(1)
    prev_close = close.shift(1)

    df['Pivot_Point'] = (prev_high + prev_low + prev_close) / 3
    df['S1'] = (df['Pivot_Point'] * 2) - prev_high
    df['S2'] = df['Pivot_Point'] - (prev_high - prev_low)
    df['R1'] = (df['Pivot_Point'] * 2) - prev_low
    df['R2'] = df['Pivot_Point'] + (prev_high - prev_low)

    df = df.replace([np.inf, -np.inf], np.nan)

    return df

all_data_indicators = {}
for name, data in all_data.items():
    print(f"\n{name}...", end=" ")
    try:
        result = hesapla_teknik_gostergeler(data)
        all_data_indicators[name] = result
        print(f"✅ {len(result)} satır")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ Göstergeler hesaplandı")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 3: VERİ HAZIRLAMA
# ============================================================================
print("="*80)
print("LEVEL 3: VERİ HAZIRLAMA (NORMALIZATION LAG ÖNCESİ)")
print("="*80)

def veri_hazirla(df, test_ratio=0.2):
    """Normalization LAG öncesi (Data leakage var)"""
    df = df.copy()

    features = ['Stochastic_K', 'Stochastic_D', 'ROC', 'Williams_R',
                'Momentum', 'Disparity_5', 'Disparity_14', 'OSCP',
                'CCI', 'RSI', 'Pivot_Point', 'S1', 'S2', 'R1', 'R2']

    # Target
    df['Next_Close'] = df['Close'].shift(-1)
    df['Target'] = (df['Next_Close'] > df['Close']).astype(int)
    df = df.iloc[:-1].copy()

    # NaN temizle
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=features + ['Target'])

    # Normalization ÖNCE
    scaler = MinMaxScaler()
    df[features] = scaler.fit_transform(df[features])

    # Lag uygula
    lagged_features = []
    for feat in features:
        lagged_col = f'{feat}_lag1'
        df[lagged_col] = df[feat].shift(1)
        lagged_features.append(lagged_col)

    df = df.dropna(subset=lagged_features)

    X = df[lagged_features].copy()
    y = df['Target'].copy()

    print(f" Veri: {len(X)} satır | Up: {y.mean()*100:.1f}%")

    # Split
    n_train = int(len(X) * (1 - test_ratio))
    X_train = X.iloc[:n_train].copy()
    X_test = X.iloc[n_train:].copy()
    y_train = y.iloc[:n_train].values
    y_test = y.iloc[n_train:].values

    print(f" Train: {len(X_train)} | Test: {len(X_test)}")

    return X_train, X_test, y_train, y_test

prepared_data = {}
for name, data in all_data_indicators.items():
    print(f"\n{name}:")
    try:
        X_train, X_test, y_train, y_test = veri_hazirla(data)
        prepared_data[name] = {
            'X_train': X_train, 'X_test': X_test,
            'y_train': y_train, 'y_test': y_test
        }
    except Exception as e:
        print(f" ❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(prepared_data)} borsa hazır")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 4: ADVANCED PARAMETER SEARCH
# ============================================================================
print("="*80)
print("LEVEL 4: ADVANCED PARAMETER SEARCH")
print("="*80)

def advanced_search_linear_svm(X_train, y_train, X_test, y_test, index_name):
    """
    🎯 Multi-Strategy Search:
    1. Two-Stage Grid Search (Coarse → Fine)
    2. Randomized Search (100 iterations)
    3. Bayesian Optimization (50 iterations)
    """

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    results = {}

    print(f"\n {'='*70}")
    print(f" STRATEGY 1: TWO-STAGE GRID SEARCH")
    print(f" {'='*70}")

    # STAGE 1: Coarse Grid
    print("\n [Stage 1] Coarse Grid...")
    coarse_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    }

    svm = SVC(kernel='linear', max_iter=50000, random_state=42)
    grid_coarse = GridSearchCV(svm, coarse_grid, cv=cv, scoring='accuracy', n_jobs=-1)
    grid_coarse.fit(X_train, y_train)

    best_C_coarse = grid_coarse.best_params_['C']
    print(f" ✓ Best C (Coarse): {best_C_coarse}")
    print(f" ✓ CV Score: {grid_coarse.best_score_:.4f}")

    # STAGE 2: Fine Grid around best
    print(f"\n [Stage 2] Fine-tuning around {best_C_coarse}...")

    # Akıllı fine grid oluştur
    if best_C_coarse < 1:
        fine_range = np.linspace(best_C_coarse * 0.5, best_C_coarse * 2, 15)
    elif best_C_coarse < 100:
        fine_range = np.linspace(best_C_coarse - 50, best_C_coarse + 50, 20)
    else:
        fine_range = np.linspace(best_C_coarse - 200, best_C_coarse + 200, 30)

    fine_range = [max(0.0001, c) for c in fine_range]  # Pozitif tut

    fine_grid = {'C': fine_range}

    grid_fine = GridSearchCV(svm, fine_grid, cv=cv, scoring='accuracy', n_jobs=-1)
    grid_fine.fit(X_train, y_train)

    best_C_fine = grid_fine.best_params_['C']
    print(f" ✓ Best C (Fine): {best_C_fine:.4f}")
    print(f" ✓ CV Score: {grid_fine.best_score_:.4f}")

    # Test
    y_pred = grid_fine.best_estimator_.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    results['two_stage'] = {
        'best_C': best_C_fine,
        'cv_score': grid_fine.best_score_,
        'test_acc': acc,
        'test_f1': f1,
        'model': grid_fine.best_estimator_
    }

    print(f" ✓ Test Acc: {acc:.4f} | F1: {f1:.4f}")

    # ========================================================================

    print(f"\n {'='*70}")
    print(f" STRATEGY 2: RANDOMIZED SEARCH")
    print(f" {'='*70}")

    print("\n [Randomized] 100 iterations with log-uniform distribution...")

    param_dist = {
        'C': loguniform(1e-4, 1e3)  # Log-uniform between 0.0001 and 1000
    }

    random_search = RandomizedSearchCV(
        svm, param_dist, n_iter=100, cv=cv,
        scoring='accuracy', n_jobs=-1, random_state=42
    )
    random_search.fit(X_train, y_train)

    best_C_random = random_search.best_params_['C']
    print(f" ✓ Best C (Random): {best_C_random:.4f}")
    print(f" ✓ CV Score: {random_search.best_score_:.4f}")

    # Test
    y_pred = random_search.best_estimator_.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    results['randomized'] = {
        'best_C': best_C_random,
        'cv_score': random_search.best_score_,
        'test_acc': acc,
        'test_f1': f1,
        'model': random_search.best_estimator_
    }

    print(f" ✓ Test Acc: {acc:.4f} | F1: {f1:.4f}")

    # ========================================================================

    print(f"\n {'='*70}")
    print(f" STRATEGY 3: BAYESIAN OPTIMIZATION")
    print(f" {'='*70}")

    print("\n [Bayesian] 50 iterations with intelligent exploration...")

    search_spaces = {
        'C': Real(1e-4, 1e3, prior='log-uniform')  # Bayesian log-uniform
    }

    bayes_search = BayesSearchCV(
        svm, search_spaces, n_iter=50, cv=cv,
        scoring='accuracy', n_jobs=-1, random_state=42
    )
    bayes_search.fit(X_train, y_train)

    best_C_bayes = bayes_search.best_params_['C']
    print(f" ✓ Best C (Bayes): {best_C_bayes:.4f}")
    print(f" ✓ CV Score: {bayes_search.best_score_:.4f}")

    # Test
    y_pred = bayes_search.best_estimator_.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    results['bayesian'] = {
        'best_C': best_C_bayes,
        'cv_score': bayes_search.best_score_,
        'test_acc': acc,
        'test_f1': f1,
        'model': bayes_search.best_estimator_
    }

    print(f" ✓ Test Acc: {acc:.4f} | F1: {f1:.4f}")

    return results

# ============================================================================
# ÇALIŞTIR
# ============================================================================

all_results = {}

for name in ['KOSPI']:  # Önce sadece KOSPI test
    print(f"\n{'='*80}")
    print(f" {name}")
    print(f"{'='*80}")

    data = prepared_data[name]

    try:
        results = advanced_search_linear_svm(
            data['X_train'],
            data['y_train'],
            data['X_test'],
            data['y_test'],
            name
        )
        all_results[name] = results

    except Exception as e:
        print(f"\n ❌ Error: {e}")
        import traceback
        traceback.print_exc()

# ============================================================================
# FINAL COMPARISON
# ============================================================================

print("\n" + "="*80)
print(" "*20 + "📊 STRATEGY COMPARISON")
print("="*80)

for name, results in all_results.items():
    print(f"\n {name}")
    print(" " + "-"*75)
    print(f" {'Strategy':<20} {'Best C':<15} {'CV Score':<12} {'Test Acc':<12} {'F1':<12}")
    print(" " + "-"*75)

    for strategy in ['two_stage', 'randomized', 'bayesian']:
        r = results[strategy]
        print(f" {strategy.upper():<20} {r['best_C']:<15.4f} {r['cv_score']:<12.4f} {r['test_acc']:<12.4f} {r['test_f1']:<12.4f}")

    # Find best
    best_strategy = max(results.items(), key=lambda x: x[1]['test_acc'])
    print(f"\n ⭐ WINNER: {best_strategy[0].upper()} (Acc: {best_strategy[1]['test_acc']:.4f})")

print("\n" + "="*80)
print("✅ ADVANCED SEARCH TAMAMLANDI")
print("="*80)

📦 Advanced Grid Search Yükleniyor...
✅ Kurulum tamamlandı!

LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)

📊 KSE100 (^KSE)... ✅ 2346 gün

📊 KOSPI (^KS11)... ✅ 2397 gün

📊 Nikkei225 (^N225)... ✅ 2382 gün

📊 SZSE (000001.SS)... ✅ 2366 gün

✅ 4 borsa çekildi

LEVEL 2: 15 TEKNİK GÖSTERGE (Table 1)

KSE100... ✅ 2346 satır

KOSPI... ✅ 2397 satır

Nikkei225... ✅ 2382 satır

SZSE... ✅ 2366 satır

✅ Göstergeler hesaplandı

LEVEL 3: VERİ HAZIRLAMA (NORMALIZATION LAG ÖNCESİ)

KSE100:
 Veri: 2325 satır | Up: 53.5%
 Train: 1860 | Test: 465

KOSPI:
 Veri: 2376 satır | Up: 52.4%
 Train: 1900 | Test: 476

Nikkei225:
 Veri: 2361 satır | Up: 53.1%
 Train: 1888 | Test: 473

SZSE:
 Veri: 2345 satır | Up: 52.8%
 Train: 1876 | Test: 469

✅ 4 borsa hazır

LEVEL 4: ADVANCED PARAMETER SEARCH

 KOSPI

 STRATEGY 1: TWO-STAGE GRID SEARCH

 [Stage 1] Coarse Grid...
 ✓ Best C (Coarse): 0.001
 ✓ CV Score: 0.5137

 [Stage 2] Fine-tuning around 0.001...
 ✓ Best C (Fine): 0.0005
 ✓ CV Score: 0.5137
 ✓ Test Acc: 0.5630

In [ ]:
"""
============================================================================
VERSİYON 4: RETURNS-BASED STATIONARY APPROACH
============================================================================
Hipotez: Financial time series non-stationary!
         Price-based değil RETURNS-BASED indicator'lar kullanmalıyız!

Değişiklikler:
- Close price → Log Returns
- Technical indicators returns bazlı
- Volatility normalization
- Stationary features
============================================================================
"""

import sys
import subprocess
print("📦 VERSİYON 4: Returns-Based Stationary Approach")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                      "yfinance", "ta", "scikit-learn", "pandas", "numpy"])

import yfinance as yf
import pandas as pd
import numpy as np
import ta
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Kurulum tamamlandı!\n")

# ============================================================================
# LEVEL 1: VERİ ÇEKME
# ============================================================================
print("="*80)
print("LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)")
print("="*80)

tickers = {
    'KSE100': '^KSE',
    'KOSPI': '^KS11',
    'Nikkei225': '^N225',
    'SZSE': '000001.SS'
}

all_data = {}
for name, ticker in tickers.items():
    print(f"\n📊 {name} ({ticker})...", end=" ")
    try:
        data = yf.download(ticker, start="2011-01-01", end="2020-09-27",
                          progress=False, auto_adjust=True)

        if data.empty:
            print("❌ VERİ YOK!")
            continue

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        data = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        data = data.dropna()

        all_data[name] = data
        print(f"✅ {len(data)} gün")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(all_data)} borsa çekildi")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 2: RETURNS-BASED STATIONARY INDICATORS
# ============================================================================
print("="*80)
print("LEVEL 2: RETURNS-BASED STATIONARY INDICATORS")
print("="*80)

def hesapla_stationary_indicators(df):
    """
    🎯 Returns-based, stationary technical indicators

    Financial time series preprocessing:
    1. Log returns (stationary)
    2. Volatility adjustment
    3. Returns-based technical indicators
    """
    df = df.copy()

    # ========================================================================
    # BASE: RETURNS & VOLATILITY
    # ========================================================================

    # 1. Log Returns (stationary!)
    df['Returns'] = np.log(df['Close'] / df['Close'].shift(1))

    # 2. Percentage Returns (alternative)
    df['Pct_Returns'] = df['Close'].pct_change()

    # 3. Volatility (20-day rolling std of returns)
    df['Volatility'] = df['Returns'].rolling(20).std()

    # 4. Volatility-adjusted returns (Sharpe-like signal)
    df['Vol_Adj_Returns'] = df['Returns'] / (df['Volatility'] + 1e-8)

    # ========================================================================
    # MOMENTUM INDICATORS (Returns-based)
    # ========================================================================

    # 5. Returns Momentum (4-period diff)
    df['Returns_Momentum'] = df['Returns'].diff(4)

    # 6. Return Acceleration (2nd derivative)
    df['Return_Accel'] = df['Returns'].diff()

    # 7. Cumulative Returns (10-day)
    df['Cum_Returns_10'] = df['Returns'].rolling(10).sum()

    # ========================================================================
    # MOVING AVERAGE INDICATORS (Returns-based)
    # ========================================================================

    # 8-9. EMA on Returns
    ema_short = df['Returns'].ewm(span=5, adjust=False).mean()
    ema_long = df['Returns'].ewm(span=14, adjust=False).mean()
    df['MACD_Returns'] = ema_short - ema_long
    df['MACD_Signal'] = df['MACD_Returns'].ewm(span=3, adjust=False).mean()

    # 10. Simple MA difference (returns)
    ma5_ret = df['Returns'].rolling(5).mean()
    ma10_ret = df['Returns'].rolling(10).mean()
    df['MA_Diff_Returns'] = ma5_ret - ma10_ret

    # ========================================================================
    # RSI & OSCILLATORS (Returns-based)
    # ========================================================================

    # 11. RSI on Returns (not price!)
    # Returns zaten +/- olduğu için daha meaningful
    returns_series = df['Returns'].fillna(0) * 100  # Scale for RSI
    df['RSI_Returns'] = ta.momentum.RSIIndicator(
        returns_series, window=14
    ).rsi()

    # 12-13. Stochastic on Returns
    # Returns'ü min-max normalize et
    returns_norm = (df['Returns'] - df['Returns'].rolling(14).min()) / \
                   (df['Returns'].rolling(14).max() - df['Returns'].rolling(14).min() + 1e-8)
    df['Stoch_Returns'] = returns_norm * 100
    df['Stoch_Returns_MA'] = df['Stoch_Returns'].rolling(3).mean()

    # ========================================================================
    # VOLATILITY INDICATORS
    # ========================================================================

    # 14. Volatility ratio (current vs historical)
    vol_ma = df['Volatility'].rolling(20).mean()
    df['Vol_Ratio'] = df['Volatility'] / (vol_ma + 1e-8)

    # 15. High-Low range (normalized by close)
    df['HL_Range'] = (df['High'] - df['Low']) / (df['Close'] + 1e-8)

    # ========================================================================
    # PRICE-BASED (but normalized) - Keep some original indicators
    # ========================================================================

    # 16. Williams %R (original but useful)
    df['Williams_R'] = ta.momentum.WilliamsRIndicator(
        df['High'], df['Low'], df['Close'], lbp=14
    ).williams_r()

    # 17. CCI (price-based but captures momentum)
    df['CCI'] = ta.trend.CCIIndicator(
        df['High'], df['Low'], df['Close'], window=20
    ).cci()

    # Inf/NaN temizleme
    df = df.replace([np.inf, -np.inf], np.nan)

    return df

all_data_indicators = {}
for name, data in all_data.items():
    print(f"\n{name}...", end=" ")
    try:
        result = hesapla_stationary_indicators(data)
        all_data_indicators[name] = result
        print(f"✅ {len(result)} satır")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ Stationary indicators hesaplandı")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 3: VERİ HAZIRLAMA (Returns-based features)
# ============================================================================
print("="*80)
print("LEVEL 3: VERİ HAZIRLAMA (Returns-based features)")
print("="*80)

def veri_hazirla_stationary(df, test_ratio=0.2):
    """
    🎯 Returns-based feature preparation

    Features: Stationary returns-based indicators
    Target: Next day return direction
    """
    df = df.copy()

    # Returns-based features
    features = [
        'Returns', 'Pct_Returns', 'Volatility', 'Vol_Adj_Returns',
        'Returns_Momentum', 'Return_Accel', 'Cum_Returns_10',
        'MACD_Returns', 'MACD_Signal', 'MA_Diff_Returns',
        'RSI_Returns', 'Stoch_Returns', 'Stoch_Returns_MA',
        'Vol_Ratio', 'HL_Range', 'Williams_R', 'CCI'
    ]

    # 1️⃣ Target: Next day return direction (positive = 1, negative = 0)
    df['Next_Return'] = df['Returns'].shift(-1)
    df['Target'] = (df['Next_Return'] > 0).astype(int)
    df = df.iloc[:-1].copy()  # Son satır NaN

    # 2️⃣ NaN temizle
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=features + ['Target'])

    # 3️⃣ Normalization ÖNCE (data leakage ama makale ile karşılaştırmak için)
    scaler = MinMaxScaler()
    df[features] = scaler.fit_transform(df[features])

    # 4️⃣ Lag uygula (T-1 features ile T predict et)
    lagged_features = []
    for feat in features:
        lagged_col = f'{feat}_lag1'
        df[lagged_col] = df[feat].shift(1)
        lagged_features.append(lagged_col)

    # 5️⃣ Lag sonrası NaN temizle
    df = df.dropna(subset=lagged_features)

    X = df[lagged_features].copy()
    y = df['Target'].copy()

    print(f" Veri: {len(X)} satır | Up: {y.mean()*100:.1f}%")

    # 6️⃣ Zaman bazlı split
    n_train = int(len(X) * (1 - test_ratio))
    X_train = X.iloc[:n_train].copy()
    X_test = X.iloc[n_train:].copy()
    y_train = y.iloc[:n_train].values
    y_test = y.iloc[n_train:].values

    print(f" Train: {len(X_train)} | Test: {len(X_test)}")

    return X_train, X_test, y_train, y_test

prepared_data = {}
for name, data in all_data_indicators.items():
    print(f"\n{name}:")
    try:
        X_train, X_test, y_train, y_test = veri_hazirla_stationary(data)
        prepared_data[name] = {
            'X_train': X_train, 'X_test': X_test,
            'y_train': y_train, 'y_test': y_test
        }
    except Exception as e:
        print(f" ❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(prepared_data)} borsa hazır (Returns-based)")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 4: SVM LINEAR KERNEL (Quick Test)
# ============================================================================
print("="*80)
print("LEVEL 4: SVM LINEAR KERNEL (Returns-based Test)")
print("="*80)

def test_svm_linear(X_train, y_train, X_test, y_test, name):
    """Quick SVM test"""

    print(f"\n {name}")
    print(" " + "-"*70)

    # Grid search
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 4, 10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
    }

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    svm = SVC(kernel='linear', max_iter=50000, random_state=42)

    grid = GridSearchCV(svm, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=0)
    grid.fit(X_train, y_train)

    # Test
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f" Best C: {grid.best_params_['C']}")
    print(f" CV Score: {grid.best_score_:.4f} ({grid.best_score_*100:.2f}%)")
    print(f" Test Acc: {acc:.4f} ({acc*100:.2f}%)")
    print(f" Precision: {prec:.4f}")
    print(f" Recall: {rec:.4f}")
    print(f" F1-Score: {f1:.4f}")

    return {
        'best_C': grid.best_params_['C'],
        'cv_score': grid.best_score_,
        'test_acc': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1
    }

results = {}

for name in prepared_data.keys():
    data = prepared_data[name]
    try:
        result = test_svm_linear(
            data['X_train'], data['y_train'],
            data['X_test'], data['y_test'],
            name
        )
        results[name] = result
    except Exception as e:
        print(f" ❌ {name}: {e}")

# ============================================================================
# SONUÇLAR
# ============================================================================
print("\n" + "="*80)
print(" "*20 + "📊 RETURNS-BASED SONUÇLAR")
print("="*80)

print(f"\n {'Index':<15} {'Best C':<10} {'CV Score':<12} {'Test Acc':<12} {'F1-Score':<12}")
print(" " + "-"*70)

for name, result in results.items():
    print(f" {name:<15} {result['best_C']:<10} {result['cv_score']:<12.4f} {result['test_acc']:<12.4f} {result['f1']:<12.4f}")

# Ortalama
if len(results) > 0:
    avg_cv = np.mean([r['cv_score'] for r in results.values()])
    avg_test = np.mean([r['test_acc'] for r in results.values()])
    avg_f1 = np.mean([r['f1'] for r in results.values()])

    print(" " + "-"*70)
    print(f" {'AVERAGE':<15} {'-':<10} {avg_cv:<12.4f} {avg_test:<12.4f} {avg_f1:<12.4f}")

# Makale karşılaştırma
print(f"\n {'='*80}")
print(" 🎯 MAKALE SONUÇLARI (Table 11)")
print(f" {'='*80}\n")
print(" Index       Linear SVM (Paper)")
print(" " + "-"*35)
print(" KSE-100     0.8519")
print(" KOSPI       0.8022")
print(" Nikkei 225  0.8022")
print(" SZSE        0.8998")

print("\n" + "="*80)
print("✅ RETURNS-BASED ANALİZ TAMAMLANDI")
print("="*80)

print("\n💡 KRİTİK FARK:")
print(" ✅ Returns-based stationary features kullanıldı")
print(" ✅ Log returns → Non-stationary problem çözüldü")
print(" ✅ Volatility adjustment eklendi")
print(" ✅ Financial time series best practices uygulandı")
print("\n🔬 Eğer sonuçlar hala düşükse:")
print(" → Veri kalitesi problemi (Yahoo Finance vs Professional data)")
print(" → Makale metodolojisinde gizli adımlar olabilir")
print(" → Farklı model (Random Forest, Neural Net) denenebilir")

📦 VERSİYON 4: Returns-Based Stationary Approach
✅ Kurulum tamamlandı!

LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)

📊 KSE100 (^KSE)... ✅ 2346 gün

📊 KOSPI (^KS11)... ✅ 2397 gün

📊 Nikkei225 (^N225)... ✅ 2382 gün

📊 SZSE (000001.SS)... ✅ 2366 gün

✅ 4 borsa çekildi

LEVEL 2: RETURNS-BASED STATIONARY INDICATORS

KSE100... ✅ 2346 satır

KOSPI... ✅ 2397 satır

Nikkei225... ✅ 2382 satır

SZSE... ✅ 2366 satır

✅ Stationary indicators hesaplandı

LEVEL 3: VERİ HAZIRLAMA (Returns-based features)

KSE100:
 Veri: 2305 satır | Up: 53.6%
 Train: 1844 | Test: 461

KOSPI:
 Veri: 2356 satır | Up: 52.5%
 Train: 1884 | Test: 472

Nikkei225:
 Veri: 2341 satır | Up: 53.0%
 Train: 1872 | Test: 469

SZSE:
 Veri: 2325 satır | Up: 52.7%
 Train: 1860 | Test: 465

✅ 4 borsa hazır (Returns-based)

LEVEL 4: SVM LINEAR KERNEL (Returns-based Test)

 KSE100
 ----------------------------------------------------------------------
 Best C: 0.001
 CV Score: 0.5401 (54.01%)
 Test Acc: 0.5184 (51.84%)
 Precision: 0.51

In [ ]:
"""
============================================================================
VERSİYON 5: MULTIPLE LAGS APPROACH
============================================================================
Hipotez: Her indicator için SADECE 1 lag değil, MULTIPLE LAGS kullanmalıyız!
         SVM zaman serisini anlamıyor, temporal pattern'i lag'lerle öğretmeliyiz!

Medium'dan öğrendiğimiz:
"SVMs do not inherently understand time series data"
→ Solution: Create multiple lag features (lag_1, lag_2, ..., lag_N)

Değişiklik:
- ESKİ: 15 indicators × 1 lag = 15 features
- YENİ: 15 indicators × 10 lags = 150 features!
============================================================================
"""

import sys
import subprocess
print("📦 VERSİYON 5: Multiple Lags (10 lags per indicator)")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                      "yfinance", "ta", "scikit-learn", "pandas", "numpy"])

import yfinance as yf
import pandas as pd
import numpy as np
import ta
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Kurulum tamamlandı!\n")

# ============================================================================
# LEVEL 1: VERİ ÇEKME
# ============================================================================
print("="*80)
print("LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)")
print("="*80)

tickers = {
    'KSE100': '^KSE',
    'KOSPI': '^KS11',
    'Nikkei225': '^N225',
    'SZSE': '000001.SS'
}

all_data = {}
for name, ticker in tickers.items():
    print(f"\n📊 {name} ({ticker})...", end=" ")
    try:
        data = yf.download(ticker, start="2011-01-01", end="2020-09-27",
                          progress=False, auto_adjust=True)

        if data.empty:
            print("❌ VERİ YOK!")
            continue

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        data = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        data = data.dropna()

        all_data[name] = data
        print(f"✅ {len(data)} gün")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(all_data)} borsa çekildi")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 2: TEKNİK GÖSTERGELER (Table 1 - Original)
# ============================================================================
print("="*80)
print("LEVEL 2: 15 TEKNİK GÖSTERGE (Table 1 - Original)")
print("="*80)

def hesapla_teknik_gostergeler(df):
    """Makalenin Table 1 formüllerine göre (original price-based)"""
    df = df.copy()

    high = df['High'].squeeze()
    low = df['Low'].squeeze()
    close = df['Close'].squeeze()

    # 1-2. Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(high, low, close,
                                             window=14, smooth_window=3)
    df['Stochastic_K'] = stoch.stoch()
    df['Stochastic_D'] = stoch.stoch_signal()

    # 3. ROC (10 period)
    df['ROC'] = ta.momentum.ROCIndicator(close, window=10).roc()

    # 4. Williams %R (14 period)
    df['Williams_R'] = ta.momentum.WilliamsRIndicator(high, low, close,
                                                       lbp=14).williams_r()

    # 5. Momentum (4 period: C_t - C_{t-4})
    df['Momentum'] = close.diff(4)

    # 6. Disparity 5
    ma5 = close.rolling(5).mean()
    df['Disparity_5'] = np.where(ma5 != 0, (close / ma5) * 100, 100)

    # 7. Disparity 14
    ma14 = close.rolling(14).mean()
    df['Disparity_14'] = np.where(ma14 != 0, (close / ma14) * 100, 100)

    # 8. OSCP (Price Oscillator)
    ma10 = close.rolling(10).mean()
    df['OSCP'] = np.where(ma5 != 0, ((ma5 - ma10) / ma5), 0)

    # 9. CCI (20 period)
    df['CCI'] = ta.trend.CCIIndicator(high, low, close, window=20).cci()

    # 10. RSI (14 period)
    df['RSI'] = ta.momentum.RSIIndicator(close, window=14).rsi()

    # 11-15. Pivot Points
    prev_high = high.shift(1)
    prev_low = low.shift(1)
    prev_close = close.shift(1)

    df['Pivot_Point'] = (prev_high + prev_low + prev_close) / 3
    df['S1'] = (df['Pivot_Point'] * 2) - prev_high
    df['S2'] = df['Pivot_Point'] - (prev_high - prev_low)
    df['R1'] = (df['Pivot_Point'] * 2) - prev_low
    df['R2'] = df['Pivot_Point'] + (prev_high - prev_low)

    df = df.replace([np.inf, -np.inf], np.nan)

    return df

all_data_indicators = {}
for name, data in all_data.items():
    print(f"\n{name}...", end=" ")
    try:
        result = hesapla_teknik_gostergeler(data)
        all_data_indicators[name] = result
        print(f"✅ {len(result)} satır")
    except Exception as e:
        print(f"❌ {e}")

print(f"\n{'='*80}")
print(f"✅ Göstergeler hesaplandı")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 3: MULTIPLE LAGS FEATURE ENGINEERING
# ============================================================================
print("="*80)
print("LEVEL 3: MULTIPLE LAGS FEATURE ENGINEERING")
print("="*80)

def veri_hazirla_multiple_lags(df, test_ratio=0.2, n_lags=10):
    """
    🎯 MULTIPLE LAGS: Her indicator için N lag oluştur

    Örnek:
    - RSI → RSI_lag1, RSI_lag2, ..., RSI_lag10
    - CCI → CCI_lag1, CCI_lag2, ..., CCI_lag10

    Total features: 15 indicators × 10 lags = 150 features!
    """
    df = df.copy()

    # Base indicators
    base_features = [
        'Stochastic_K', 'Stochastic_D', 'ROC', 'Williams_R',
        'Momentum', 'Disparity_5', 'Disparity_14', 'OSCP',
        'CCI', 'RSI', 'Pivot_Point', 'S1', 'S2', 'R1', 'R2'
    ]

    # 1️⃣ Target: Next day direction
    df['Next_Close'] = df['Close'].shift(-1)
    df['Target'] = (df['Next_Close'] > df['Close']).astype(int)
    df = df.iloc[:-1].copy()

    # 2️⃣ NaN temizle
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=base_features + ['Target'])

    # 3️⃣ ⚠️ NORMALIZATION ÖNCE (data leakage var ama makale ile karşılaştırmak için)
    scaler = MinMaxScaler()
    df[base_features] = scaler.fit_transform(df[base_features])

    # 4️⃣ 🎯 MULTIPLE LAGS OLUŞTUR
    print(f" Creating {n_lags} lags for each of {len(base_features)} indicators...")

    lagged_features = []
    for indicator in base_features:
        for lag in range(1, n_lags + 1):
            lagged_col = f'{indicator}_lag{lag}'
            df[lagged_col] = df[indicator].shift(lag)
            lagged_features.append(lagged_col)

    print(f" Total lagged features created: {len(lagged_features)}")

    # 5️⃣ Lag sonrası NaN temizle (en büyük lag kadar satır kaybedilir)
    df = df.dropna(subset=lagged_features)

    X = df[lagged_features].copy()
    y = df['Target'].copy()

    print(f" Veri: {len(X)} satır | Features: {X.shape[1]} | Up: {y.mean()*100:.1f}%")

    # 6️⃣ Zaman bazlı split
    n_train = int(len(X) * (1 - test_ratio))
    X_train = X.iloc[:n_train].copy()
    X_test = X.iloc[n_train:].copy()
    y_train = y.iloc[:n_train].values
    y_test = y.iloc[n_train:].values

    print(f" Train: {len(X_train)} | Test: {len(X_test)}")

    return X_train, X_test, y_train, y_test

prepared_data = {}
for name, data in all_data_indicators.items():
    print(f"\n{name}:")
    try:
        X_train, X_test, y_train, y_test = veri_hazirla_multiple_lags(data, n_lags=10)
        prepared_data[name] = {
            'X_train': X_train, 'X_test': X_test,
            'y_train': y_train, 'y_test': y_test
        }
    except Exception as e:
        print(f" ❌ {e}")

print(f"\n{'='*80}")
print(f"✅ {len(prepared_data)} borsa hazır (Multiple lags)")
print(f"{'='*80}\n")

# ============================================================================
# LEVEL 4: SVM LINEAR KERNEL
# ============================================================================
print("="*80)
print("LEVEL 4: SVM LINEAR KERNEL (Multiple Lags Test)")
print("="*80)

def test_svm_linear(X_train, y_train, X_test, y_test, name):
    """SVM Linear kernel test"""

    print(f"\n {name}")
    print(" " + "-"*70)

    # Grid search
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 4, 10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
    }

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    svm = SVC(kernel='linear', max_iter=50000, random_state=42)

    grid = GridSearchCV(svm, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=0)
    grid.fit(X_train, y_train)

    # Test
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f" Best C: {grid.best_params_['C']}")
    print(f" CV Score: {grid.best_score_:.4f} ({grid.best_score_*100:.2f}%)")
    print(f" Test Acc: {acc:.4f} ({acc*100:.2f}%)")
    print(f" Precision: {prec:.4f}")
    print(f" Recall: {rec:.4f}")
    print(f" F1-Score: {f1:.4f}")

    return {
        'best_C': grid.best_params_['C'],
        'cv_score': grid.best_score_,
        'test_acc': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1
    }

results = {}

for name in prepared_data.keys():
    data = prepared_data[name]
    try:
        result = test_svm_linear(
            data['X_train'], data['y_train'],
            data['X_test'], data['y_test'],
            name
        )
        results[name] = result
    except Exception as e:
        print(f" ❌ {name}: {e}")

# ============================================================================
# SONUÇLAR
# ============================================================================
print("\n" + "="*80)
print(" "*20 + "📊 MULTIPLE LAGS SONUÇLAR")
print("="*80)

print(f"\n {'Index':<15} {'Best C':<10} {'CV Score':<12} {'Test Acc':<12} {'F1-Score':<12}")
print(" " + "-"*70)

for name, result in results.items():
    print(f" {name:<15} {result['best_C']:<10} {result['cv_score']:<12.4f} {result['test_acc']:<12.4f} {result['f1']:<12.4f}")

# Ortalama
if len(results) > 0:
    avg_cv = np.mean([r['cv_score'] for r in results.values()])
    avg_test = np.mean([r['test_acc'] for r in results.values()])
    avg_f1 = np.mean([r['f1'] for r in results.values()])

    print(" " + "-"*70)
    print(f" {'AVERAGE':<15} {'-':<10} {avg_cv:<12.4f} {avg_test:<12.4f} {avg_f1:<12.4f}")

# Önceki versiyonlarla karşılaştırma
print(f"\n {'='*80}")
print(" 📈 VERSİYON KARŞILAŞTIRMASI")
print(f" {'='*80}\n")

print(" Versiyon                        Avg Test Acc")
print(" " + "-"*50)
print(" V1: Norm LAG öncesi (1 lag)    0.5630")
print(" V2: Target same-day             0.5147")
print(" V3: Pivot double-lag fix        0.5630")
print(" V4: Returns-based               0.5323")
if len(results) > 0:
    print(f" V5: Multiple lags (10)          {avg_test:.4f}")

# Makale karşılaştırma
print(f"\n {'='*80}")
print(" 🎯 MAKALE SONUÇLARI (Table 11)")
print(f" {'='*80}\n")
print(" Index       Linear SVM (Paper)")
print(" " + "-"*35)
print(" KSE-100     0.8519")
print(" KOSPI       0.8022")
print(" Nikkei 225  0.8022")
print(" SZSE        0.8998")

print("\n" + "="*80)
print("✅ MULTIPLE LAGS ANALİZ TAMAMLANDI")
print("="*80)

print("\n💡 KRİTİK FARK:")
print(" ✅ Her indicator için 10 lag kullanıldı (150 features)")
print(" ✅ Temporal pattern'leri yakalamak için SVM'e 'geçmiş' verildi")
print(" ✅ Medium makalesindeki 'lag features' yaklaşımı uygulandı")
print("\n🔬 Sonuçların yorumu:")
print(" → Eğer accuracy ARTTI: Multiple lags işe yarıyor! ✅")
print(" → Eğer accuracy AYNI: Problem başka yerde (veri kalitesi)")
print(" → Eğer accuracy DÜŞTÜ: Overfitting (150 feature çok fazla)")

📦 VERSİYON 5: Multiple Lags (10 lags per indicator)
✅ Kurulum tamamlandı!

LEVEL 1: VERİ ÇEKME (2011-01-01 to 2020-09-27)

📊 KSE100 (^KSE)... ✅ 2346 gün

📊 KOSPI (^KS11)... ✅ 2397 gün

📊 Nikkei225 (^N225)... ✅ 2382 gün

📊 SZSE (000001.SS)... ✅ 2366 gün

✅ 4 borsa çekildi

LEVEL 2: 15 TEKNİK GÖSTERGE (Table 1 - Original)

KSE100... ✅ 2346 satır

KOSPI... ✅ 2397 satır

Nikkei225... ✅ 2382 satır

SZSE... ✅ 2366 satır

✅ Göstergeler hesaplandı

LEVEL 3: MULTIPLE LAGS FEATURE ENGINEERING

KSE100:
 Creating 10 lags for each of 15 indicators...
 Total lagged features created: 150
 Veri: 2316 satır | Features: 150 | Up: 53.6%
 Train: 1852 | Test: 464

KOSPI:
 Creating 10 lags for each of 15 indicators...
 Total lagged features created: 150
 Veri: 2367 satır | Features: 150 | Up: 52.4%
 Train: 1893 | Test: 474

Nikkei225:
 Creating 10 lags for each of 15 indicators...
 Total lagged features created: 150
 Veri: 2352 satır | Features: 150 | Up: 53.0%
 Train: 1881 | Test: 471

SZSE:
 Creating 10 l